In [ ]:
import os
import io
import traceback
import contextlib
import pandas as pd
import asyncio
from dotenv import load_dotenv

# Google ADK Imports
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.genai import types

# =====================================================================
# 1. SETUP & AUTH
# =====================================================================

load_dotenv()
try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found.")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Configuration Complete.")
except Exception as e:
    print(f"❌ Auth Error: {e}")

# Retry config to handle transient API issues
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[400, 429, 500, 503, 504],
)

# =====================================================================
# 2. PERSISTENT PYTHON KERNEL (LAYER 3)
# =====================================================================

class PersistentPythonSession:
    """
    A persistent Python execution environment.

    - Uses a shared namespace so variables (e.g., df) survive between calls.
    - Captures stdout/stderr and returns it as a string.
    """

    def __init__(self):
        self.namespace = {
            "pd": pd,
            "__builtins__": __builtins__,
        }

    def execute(self, script: str) -> str:
        """
        Execute arbitrary Python code in a persistent namespace and
        capture stdout/stderr as a string.
        """
        output_buffer = io.StringIO()
        try:
            with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
                exec(script, self.namespace, self.namespace)
            result = output_buffer.getvalue()
            if not result:
                result = "(Success. No output printed. Remember to print results!)"
            return result
        except Exception:
            return f"Runtime Error: {traceback.format_exc()}"


# Initialize Global Session
session = PersistentPythonSession()


def run_python_kernel(script: str) -> str:
    """
    Tool used by the DataEngineer agent to execute Python code
    inside the persistent session.
    """
    print("\n🧪 [ENGINEER -> KERNEL] Executing script:")
    print("-" * 80)
    print(script)
    print("-" * 80)

    result = session.execute(script)

    print("\n📤 [KERNEL OUTPUT]")
    print("-" * 80)
    print(result)
    print("-" * 80)

    return result

# =====================================================================
# 3. DATA ENGINEER AGENT (LAYER 2)
# =====================================================================

ENGINEER_PROMPT = """
You are a Senior Python Data Engineer working in a persistent Python environment.

CONTEXT:
- You receive high-level analytical requests from a "Lead Analyst" manager agent.
- The manager describes the datasets and data sources based on the user's request.
- Datasets may be CSV files, tables, or other data sources. Do not assume any specific
  filename or schema. Always rely on what the manager explicitly tells you.

GOAL:
- Translate the manager's requests into robust, efficient Pandas code and run it.

RULES:

1. Persistent State:
   - The Python environment is stateful.
   - If you create a DataFrame like `df` or `sales_df`, it will still exist later.
   - Only load or create data structures when explicitly requested by the manager.
   - Respect the names the manager gives you (e.g. "load file X into df").

2. Tool Usage:
   - To execute ANY Python code, ALWAYS use the `run_python_kernel` tool.
   - Pass a complete, runnable Python script string.
   - If the manager says "assume the file is in the current folder", use a relative path.

3. Error Handling:
   - If the kernel output starts with 'Runtime Error:', treat that as a failure.
   - Think about what probably went wrong (e.g. wrong column name, wrong path, type issue).
   - Propose a fix and call `run_python_kernel` again with corrected code.
   - Explain in simple terms what changed and why.

4. Output Formatting:
   - Before executing code, briefly explain (1–3 sentences) what you will do,
     referring to the dataset(s) and columns by the names given by the manager.
   - In the code, use print() for:
       - df.head()
       - df.info() or df.dtypes
       - groupby / aggregation results
       - correlation matrices
       - any key statistics or intermediate results
   - After seeing the kernel output (conceptually), summarize the main findings
     in clear natural language. You do NOT need to repeat every line of output,
     just the important insights.

5. Scope of Work:
   - You are not responsible for high-level business hypotheses.
   - Focus on:
       - loading/merging datasets requested by the manager
       - cleaning and transforming data
       - grouping, aggregating, computing correlations
       - running the specific checks/tests requested
   - Always stay aligned with the manager's instructions and the user's described problem.
"""

engineer_agent = LlmAgent(
    name="DataEngineer",
    model=Gemini(model="gemini-2.5-pro", retry_options=retry_config),
    instruction=ENGINEER_PROMPT,
    tools=[run_python_kernel],
)

ENGINEER_APP_NAME = "data_engineer_app"
ENGINEER_USER_ID = "internal_user"
ENGINEER_SESSION_ID = "internal_engineer_session"

# Runner for the Engineer (stateful)
engineer_runner = InMemoryRunner(agent=engineer_agent, app_name=ENGINEER_APP_NAME)

# =====================================================================
# 4. BRIDGE TOOL: MANAGER → ENGINEER
# =====================================================================

async def ask_data_engineer(request: str) -> str:
    """
    Tool that the LeadAnalyst uses to send natural-language instructions
    to the DataEngineer agent.

    Internally:
    - Ensures a persistent session for the DataEngineer.
    - Streams all Engineer events (text + tool calls).
    - Aggregates textual responses into a single string.
    """
    print(f"\n📢 [MANAGER -> ENGINEER]: {request}")

    try:
        # Ensure the Engineer session exists (idempotent)
        try:
            await engineer_runner.session_service.create_session(
                app_name=ENGINEER_APP_NAME,
                user_id=ENGINEER_USER_ID,
                session_id=ENGINEER_SESSION_ID,
            )
            print(f"🧵 [ENGINEER SESSION] Created/using session '{ENGINEER_SESSION_ID}'.")
        except Exception as e:
            # If it already exists, that's fine
            print(f"ℹ️ [ENGINEER SESSION] Using existing session '{ENGINEER_SESSION_ID}': {e}")

        user_msg = types.UserContent(parts=[types.Part(text=request)])

        full_response_text: list[str] = []

        # Stream of events coming from the Engineer
        async for event in engineer_runner.run_async(
            user_id=ENGINEER_USER_ID,
            session_id=ENGINEER_SESSION_ID,
            new_message=user_msg,
        ):
            if event.content and event.content.parts:
                role = getattr(event.content, "role", "model")
                print(f"\n🤖 [ENGINEER EVENT] role={role}")

                for part in event.content.parts:
                    # Text from Engineer
                    if getattr(part, "text", None):
                        print("   🗣️ ENGINEER TEXT:")
                        print("   " + part.text.replace("\n", "\n   "))
                        full_response_text.append(part.text)

                    # Tool calls (e.g. run_python_kernel)
                    if getattr(part, "function_call", None):
                        fc = part.function_call
                        print("   🛠️ ENGINEER TOOL CALL:")
                        print(f"      name: {fc.name}")
                        print(f"      args: {fc.args}")

        output_text = "".join(full_response_text).strip()

        if not output_text:
            output_text = "(Task completed by Engineer. Check logs above for kernel output and messages.)"

        print(f"\n📉 [ENGINEER -> MANAGER]: Final aggregated response delivered.")
        return output_text

    except Exception as e:
        error_msg = f"Data Engineer System Error: {str(e)}"
        print(f"❌ [ENGINEER ERROR]: {error_msg}")
        return error_msg

# =====================================================================
# 5. LEAD ANALYST AGENT (LAYER 1)
# =====================================================================

MANAGER_PROMPT = """
You are the Lead Data Analyst. You manage a specialized "Data Engineer" who writes and executes code for you.
You do not write code yourself; you think strategically and scientifically.

CONTEXT:
- The human user will describe:
    - One or more datasets (e.g. CSV files, tables, views)
    - The business question or analytical goal
- You must always ground your reasoning in the datasets and goals described
  in the USER'S QUERY, not in any fixed example dataset.
- You have a tool `ask_data_engineer(request="...")` that can translate your analytical
  requests into actual code and run it on the data.

GENERAL WORKFLOW (ADAPTED TO ANY DATASET):

PHASE 1 – EXPLORATION
- From the USER'S QUERY, identify:
  - Which dataset(s) should be loaded (filenames, tables, etc.).
  - The likely target metrics and key dimensions.
- Use ask_data_engineer(request="...") to:
  - Load the dataset(s) the user mentioned into appropriately named DataFrames (e.g. df, df_sales).
  - Show a small sample (head), the schema (dtypes/info), and basic descriptive stats.
- From the Engineer's summary, clarify:
  - The main time dimension(s) (e.g. date, day, month), if any.
  - The main outcome metrics (e.g., revenue, conversions, TTI, error rate, etc.).
  - Any important categorical segments (e.g., country, device, channel, segment, endpoint).

PHASE 2 – HYPOTHESIS FORMULATION
- Based on the exploration and the user’s goal, define exactly 3 hypotheses.
- Use the following format:
  - H1: [short hypothesis statement about a potential driver or relationship]
  - H2: ...
  - H3: ...
- Each hypothesis should explain a potential DRIVER of the patterns in the dataset(s)
  the user cares about (e.g. segments, time effects, correlations between metrics).

PHASE 3 – TESTING
- For each hypothesis (H1, H2, H3):
  - Use ask_data_engineer(request="...") to request specific computations.
  - Examples of useful operations (adapt to the actual data and goal):
      - groupby on segments (e.g., endpoint, region, device, channel)
      - correlation between metrics (e.g. traffic vs. errors, TTI vs. conversion)
      - time-based analyses (trends, before/after periods, weekday vs weekend, etc.)
  - Ask the Engineer to produce compact tables and relevant statistics, not huge dumps.
- After each Engineer response:
  - Interpret the evidence.
  - Decide whether each hypothesis is:
      - Supported
      - Not supported
      - Inconclusive
  - Briefly justify your decision, referencing the Engineer's results.

PHASE 4 – SYNTHESIS & REPORT
- Summarize in a short analytical report:
  - The key drivers or relationships you discovered in the dataset(s) the user described.
  - Which hypotheses were accepted, rejected, or remained inconclusive.
  - Any surprising or non-obvious insights that emerged.
- Use clear headings and concise, business-oriented language.

TOOL USAGE:
- Treat ask_data_engineer(request="...") as your interface to the underlying Python environment.
- Do NOT write code yourself; always express what you need in natural language.
- Keep your requests specific and grounded in the columns, metrics, and datasets
  mentioned by the user.

REQUIREMENT:
At the very end of your response, always include a section:

### 🔭 Future Exploration

In this section, list 3–5 concrete next questions or analyses that could be run
in future iterations, tailored to the user's data and goal
(e.g., "Test seasonality by month", "Compare pre- and post-release performance",
      "Model uplift after a campaign", "Drill into a specific endpoint or segment").
"""

lead_analyst = LlmAgent(
    name="LeadAnalyst",
    model=Gemini(model="gemini-2.5-pro", retry_options=retry_config),
    instruction=MANAGER_PROMPT,
    tools=[ask_data_engineer],
)

# =====================================================================
# 6. EXECUTION
# =====================================================================

# 👉 EXAMPLE user query.
# You can change this to any other dataset/problem;
# the prompts are now fully generic.
user_query = """
I have a CSV file named 'Euro_Daily_Trends.csv' in the current folder.
It contains daily metrics for different European entities.

I want to understand the drivers behind the main trends in this dataset.
Please follow your exploration → hypotheses → testing → synthesis workflow,
and focus on finding non-obvious relationships rather than just describing the data.
"""

print("--- 🧠 Lead Analyst Team Started ---")
runner = InMemoryRunner(agent=lead_analyst, app_name="lead_analyst_app")



await runner.run_debug(user_query)

✅ Configuration Complete.
--- 🧠 Lead Analyst Team Started ---

 ### Created new session: debug_session_id

User > 
I have a CSV file named 'Euro_Daily_Trends.csv' in the current folder.
It contains daily metrics for different European entities.

I want to understand the drivers behind the main trends in this dataset.
Please follow your exploration → hypotheses → testing → synthesis workflow,
and focus on finding non-obvious relationships rather than just describing the data.



LeadAnalyst > Excellent. I will now begin a systematic analysis of the `Euro_Daily_Trends.csv` dataset to identify the key drivers of its main trends.

### Phase 1: Exploration

First, I need to understand the structure and content of the dataset. I will ask my Data Engineer to load the file and provide a basic overview, including the first few rows, the data types of the columns, and summary statistics. This will help me identify the key metrics and dimensions for our analysis.

📢 [MANAGER -> ENGINEER]: Load the 'Euro_Daily_Trends.csv' file into a pandas DataFrame. Then, please provide the following:
1.  The first 5 rows of the DataFrame.
2.  The data types of each column (info).
3.  A descriptive statistical summary of the numerical columns.
🧵 [ENGINEER SESSION] Created/using session 'internal_engineer_session'.



🤖 [ENGINEER EVENT] role=model
   🗣️ ENGINEER TEXT:
   Okay, I will load the `Euro_Daily_Trends.csv` file into a pandas DataFrame. Then, I will display the first 5 rows, the data type information for each column, and a statistical summary of the numerical columns.
   🛠️ ENGINEER TOOL CALL:
      name: run_python_kernel
      args: {'script': '\nimport pandas as pd\n\n# Load the dataset\ntry:\n    df = pd.read_csv(\'Euro_Daily_Trends.csv\')\n    print("DataFrame loaded successfully.\n")\n    # 1. Display the first 5 rows\n    print("First 5 rows of the DataFrame:")\n    print(df.head())\n    print("\\n" + "="*50 + "\\n")\n\n    # 2. Display the data types of each column\n    print("DataFrame Info:")\n    df.info()\n    print("\\n" + "="*50 + "\\n")\n\n    # 3. Display descriptive statistics\n    print("Descriptive Statistics for numerical columns:")\n    print(df.describe())\n\nexcept FileNotFoundError:\n    print("Error: \'Euro_Daily_Trends.csv\' not found. Please ensure the file is in


🤖 [ENGINEER EVENT] role=model
   🗣️ ENGINEER TEXT:
   It appears my previous code failed due to a `SyntaxError`. The error was caused by an unclosed string in a `print` statement. I have fixed this syntax issue and will now rerun the script to load the `Euro_Daily_Trends.csv` data and provide the requested summaries.
   🛠️ ENGINEER TOOL CALL:
      name: run_python_kernel
      args: {'script': '\nimport pandas as pd\n\n# Load the dataset\ntry:\n    df = pd.read_csv(\'Euro_Daily_Trends.csv\')\n    print("DataFrame loaded successfully.")\n    print("\\n" + "="*50 + "\\n")\n\n    # 1. Display the first 5 rows\n    print("First 5 rows of the DataFrame:")\n    print(df.head())\n    print("\\n" + "="*50 + "\\n")\n\n    # 2. Display the data types of each column\n    print("DataFrame Info:")\n    df.info()\n    print("\\n" + "="*50 + "\\n")\n\n    # 3. Display descriptive statistics\n    print("Descriptive Statistics for numerical columns:")\n    print(df.describe())\n\nexcept FileNotFoundE


🤖 [ENGINEER EVENT] role=model
   🗣️ ENGINEER TEXT:
   The `Euro_Daily_Trends.csv` file has been successfully loaded. Here is a summary of the data:
   
   *   **First 5 Rows**: The dataset includes daily records for cities like Amsterdam, Madrid, and Berlin. The columns track the date, city, country, day of the week, weather, average temperature, commute time, primary transport mode, coffee consumption, work productivity, and whether there was a public transit strike.
   
   *   **Data Types**: The DataFrame contains 30 rows and 11 columns with no missing values. The data types are a mix of `object` (for text data like City and Country), `int64` (for Temperature and Commute Time), `float64` (for Coffee Cups and Productivity), and `bool` (for the Strike Flag).
   
   *   **Statistical Summary**:
       *   **Temperature (`Avg_Temp_C`)**: The average temperature is 11.5°C, with a range from 2°C to 23°C.
       *   **Commute Time (`Avg_Commute_Time_Mins`)**: The average commute time is a

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2, model: gemini-2.5-pro\nPlease retry in 32.726169318s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-pro', 'location': 'g